In [1]:
def get_lines(file_type='sample'):
    '''
    Read in the lines of today's sample/input file line by line. 
    Assumes the file is in folder called 'inputs/'
    
    Parameters
    ----------
    file_type : str
        Either sample or input
    
    Returns
    -------
    list of inputs stripped of whitespace
    '''
    import datetime
    day = str(datetime.datetime.today().day).zfill(2)
    filename = f'inputs/{day}-{file_type}.txt'
    try:
        with open(filename,'r') as file:
            lines = [line.strip() for line in file.readlines()]
        return lines
    except:
        print(filename+' does not exist')

In [2]:
sample = get_lines('sample')

In [3]:
sample

['R 4', 'U 4', 'L 3', 'D 1', 'R 4', 'D 1', 'L 5', 'R 2']

In [4]:
def parse_move(move):
    split = move.split(' ')
    direction = split[0]
    steps = int(split[1])
    return direction, steps

In [5]:
parse_move('R 4')

('R', 4)

Assume that the head and tail start in the same position, (0,0). Keep track of the position of the head and the tail with coordinates.

In [6]:
from collections import namedtuple

In [7]:
Point = namedtuple('Point','row col')

In [8]:
head = Point(0,0)
tail = Point(0,0)

In [9]:
head

Point(row=0, col=0)

**`After each step, you'll need to update the position of the tail if the step means the head is no longer adjacent to the tail.`**

In [10]:
def step_up(point, steps=1):
    loc = Point(point.row+steps, point.col)
    return loc

def step_down(point, steps=1):
    loc = Point(point.row-steps, point.col)
    return loc

def step_right(point, steps=1):
    loc = Point(point.row, point.col+steps)
    return loc

def step_left(point, steps=1):
    loc = Point(point.row, point.col-steps)
    return loc

In [11]:
def is_adjacent(head,tail):
    if (abs(head.row-tail.row) <= 1) & (abs(head.col-tail.col) <= 1):
        return True
    return False

**`If the head is ever two steps directly up, down, left, or right from the tail, the tail must also move one step in that direction;
if the head and tail aren't touching and aren't in the same row or column, the tail always moves one step diagonally to keep up`**

In [12]:
def check_diagonal(head,tail):
    if (abs(head.row-tail.row) > 0) & (abs(head.col-tail.col) > 0):
#         print('is diagonal')
        return True
    return False

### TRICKY
In part 1, the head and tail could not be 2 diagonals apart from each other, but in part 2 a knot CAN be 2 diagonals away from the previous knot. Deal with that edge case.

In [13]:
def big_step_diagonal(previous, current):
    if previous.row-current.row > 0:
        current = step_up(current)
        if previous.col-current.col > 0:
            current = step_right(current)
        else:
            current = step_left(current)
    else:
        current = step_down(current)
        if previous.col-current.col > 0:
            current = step_right(current)
        else:
            current = step_left(current)
    return current

In [14]:
def step_diagonal(previous,current):
    if (abs(previous.row)-abs(current.row))==(abs(previous.col)-abs(current.col)):
        return big_step_diagonal(previous, current)
    if previous.row > current.row:
        current = step_up(current)
    else:
        current = step_down(current)
    if previous.col > current.col:
        current = step_right(current)
    else:
        current = step_left(current)
    return current

In [15]:
def update_knot(previous,current):
    '''assumes NOT adjacent, will always move the tail'''
    if check_diagonal(previous,current):
        return step_diagonal(previous,current)
    if previous.row > current.row:
        return step_up(current)
    elif previous.row < current.row:
        return step_down(current)
    if previous.col > current.col:
        return step_right(current)
    elif previous.col < current.col:
        return step_left(current)
    print('update knot error')
    return None

        

In [16]:
def make_tail_step(previous,tail):
    locs_visited.add(tail)
    if not is_adjacent(previous,tail):
        tail = update_knot(previous,tail)
#         print('tail moved to ',tail)
    else:
        pass
#         print('tail not moved')
    locs_visited.add(tail)
    return tail
    

Minor improvement from Part 1 to the steps for the head knot: use a dictionary of the functions rather than a series of if-else statements.

In [17]:
def make_head_step(head, direction):
    direction_moves = {'U':step_up,'D':step_down,'R':step_right,'L':step_left}
    head = direction_moves[direction](head)
    return head

In [18]:
def make_knot_step(previous, current):
    if not is_adjacent(previous, current):
        current = update_knot(previous, current)
    else:
        pass
    return current

In [19]:
def initialize_tail_locations():
    return set()

In [20]:
def initialize_point_state():
    return (Point(0,0),Point(0,0))

In [21]:
def get_moves(file_lines):
    return [parse_move(move) for move in file_lines]

### Initialize states

In this version, the rope has 10 knots to keep track of. They all start in the same location.

The head of this rope is `knots[0]` and the tail is `knots[9]`.

In [22]:
head_moves = get_moves(get_lines('sample'))
head_moves

[('R', 4),
 ('U', 4),
 ('L', 3),
 ('D', 1),
 ('R', 4),
 ('D', 1),
 ('L', 5),
 ('R', 2)]

In [23]:
knots = [Point(0,0) for knot in range(10)]
locs_visited = initialize_tail_locations()
knots, locs_visited

([Point(row=0, col=0),
  Point(row=0, col=0),
  Point(row=0, col=0),
  Point(row=0, col=0),
  Point(row=0, col=0),
  Point(row=0, col=0),
  Point(row=0, col=0),
  Point(row=0, col=0),
  Point(row=0, col=0),
  Point(row=0, col=0)],
 set())

In [24]:
def move_rope(knots, move):
    direction, steps = move
    for step in range(steps):
        knots[0] = make_head_step(knots[0], direction)
        for i in range(1,9):
            knots[i] = make_knot_step(knots[i-1],knots[i])
        knots[9] = make_tail_step(knots[8],knots[9])
    return knots
        

In [25]:
for move in head_moves:
    knots = move_rope(knots, move)
    print(move)
    for knot in knots:
        print('\t',knot)
    print('\n')

('R', 4)
	 Point(row=0, col=4)
	 Point(row=0, col=3)
	 Point(row=0, col=2)
	 Point(row=0, col=1)
	 Point(row=0, col=0)
	 Point(row=0, col=0)
	 Point(row=0, col=0)
	 Point(row=0, col=0)
	 Point(row=0, col=0)
	 Point(row=0, col=0)


('U', 4)
	 Point(row=4, col=4)
	 Point(row=3, col=4)
	 Point(row=2, col=4)
	 Point(row=2, col=3)
	 Point(row=2, col=2)
	 Point(row=1, col=1)
	 Point(row=0, col=0)
	 Point(row=0, col=0)
	 Point(row=0, col=0)
	 Point(row=0, col=0)


('L', 3)
	 Point(row=4, col=1)
	 Point(row=4, col=2)
	 Point(row=3, col=3)
	 Point(row=2, col=3)
	 Point(row=2, col=2)
	 Point(row=1, col=1)
	 Point(row=0, col=0)
	 Point(row=0, col=0)
	 Point(row=0, col=0)
	 Point(row=0, col=0)


('D', 1)
	 Point(row=3, col=1)
	 Point(row=4, col=2)
	 Point(row=3, col=3)
	 Point(row=2, col=3)
	 Point(row=2, col=2)
	 Point(row=1, col=1)
	 Point(row=0, col=0)
	 Point(row=0, col=0)
	 Point(row=0, col=0)
	 Point(row=0, col=0)


('R', 4)
	 Point(row=3, col=5)
	 Point(row=3, col=4)
	 Point(row=3, col=3)
	

In [26]:
len(locs_visited)

1

### Now run this on the input

In [29]:
head_moves = get_moves(get_lines('input'))
head, tail = initialize_point_state()
locs_visited = initialize_tail_locations()

In [30]:
for move in head_moves:
    knots = move_rope(knots, move)
#     print(move)
#     for knot in knots:
#         print('\t',knot)
#     print('\n')
print(len(locs_visited))

2593
